In [ ]:
import requests
import pandas as pd

def fetch_crypto_data(symbol, days=90):
    """Fetch historical price data for a given cryptocurrency symbol"""
    url = f"https://api.coingecko.com/api/v3/coins/{symbol}/market_chart?vs_currency=usd&days={days}"
    response = requests.get(url)
    data = response.json()

    # Convert to DataFrame
    df = pd.DataFrame(data["prices"], columns=["timestamp", "price"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")

    return df

# Example usage
coin = "bitcoin"  # Change to any symbol like 'ethereum', 'cardano'
df = fetch_crypto_data(coin)
print(df.head())


                timestamp         price
0 2024-11-24 05:05:36.582  98300.808029
1 2024-11-24 06:10:46.669  98321.946200
2 2024-11-24 07:02:26.079  98473.089148
3 2024-11-24 08:08:14.166  98553.767530
4 2024-11-24 09:01:43.748  98207.045810


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

def train_model(df):
    """Train XGBoost model on given crypto price data"""
    df["MA_3"] = df["price"].rolling(window=3).mean()
    df["MA_7"] = df["price"].rolling(window=7).mean()
    df["Price_Change_%"] = df["price"].pct_change() * 100
    df.dropna(inplace=True)

    X = df[["MA_3", "MA_7", "Price_Change_%"]]
    y = df["price"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    model = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=3)
    model.fit(X_train, y_train)

    return model, X

# Train model on dynamically fetched data
model, X = train_model(df)


In [ ]:
def predict_next_price(model, X):
    """Predict the next hour's price"""
    latest_features = X.iloc[-1].values.reshape(1, -1)
    next_price = model.predict(latest_features)
    return next_price[0]

# Get prediction
predicted_price = predict_next_price(model, X)
print(f"💰 Predicted Next Price: {predicted_price}")


💰 Predicted Next Price: 96432.09375


In [ ]:
coin_symbol = input("Enter the crypto symbol (e.g., bitcoin, ethereum): ").lower()
df = fetch_crypto_data(coin_symbol)
model, X = train_model(df)
predicted_price = predict_next_price(model, X)

print(f"🔮 Predicted Next Price for {coin_symbol.upper()}: ${predicted_price:.2f}")


Enter the crypto symbol (e.g., bitcoin, ethereum): ethereum
🔮 Predicted Next Price for ETHEREUM: $2598.93
